# 🚀 45-Minute Hands-On: LLMs with Hugging Face (Colab/Jupyter)

**Last updated:** 2025-09-04  

## Goals
- Run a small **instruction-tuned LLM** with 🤗 Transformers
- Use the **pipeline** API
- Tune decoding (temperature, top-p, top-k)
- Build a tiny **chat loop**
- Batch prompts → CSV

# 1) Install dependencies

In [1]:
!pip -q install -U transformers accelerate datasets sentencepiece pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 69.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.


# 2) Imports & device

In [2]:
import torch, time
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


## Model choice
We try **TinyLlama/TinyLlama-1.1B-Chat-v1.0** and fall back to **distilgpt2** if needed.

# 3) Load model

In [3]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
fallback_model_id = "distilgpt2"

def load_model(model_name):
    try:
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None
        )
        return tok, mdl, model_name
    except Exception as e:
        print("Primary failed:", e, "\nFalling back to", fallback_model_id)
        tok = AutoTokenizer.from_pretrained(fallback_model_id, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            fallback_model_id,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None
        )
        return tok, mdl, fallback_model_id

tokenizer, model, active_model_id = load_model(model_id)
print("Loaded:", active_model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Loaded: TinyLlama/TinyLlama-1.1B-Chat-v1.0


## Quickstart with `pipeline`
# 4) Text generation quickstart

In [4]:
gen = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Prompt for Task 1 (Healthcare AI Agents)
prompt = "How can multiple GPT-powered agents collaboratively summarize medical text, refine research articles, and sanitize sensitive healthcare data?"

out = gen(prompt, max_new_tokens=120, do_sample=True, temperature=0.7, top_p=0.9,
          pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]

print("Model in use:", active_model_id)
print("\nGenerated Output:\n", out)

Device set to use cuda:0


Model in use: TinyLlama/TinyLlama-1.1B-Chat-v1.0

Generated Output:
 How can multiple GPT-powered agents collaboratively summarize medical text, refine research articles, and sanitize sensitive healthcare data? Answer according to: Research has shown that patients’ medical histories are the most valuable information for healthcare providers to understand their patients. However, medical histories are often sensitive and confidential, making them difficult to share with multiple parties.
GPT-powered agents can be used to summarize medical text and refine research articles to make them more useful for researchers. They can also sanitize sensitive healthcare data to make it safe for researchers to use.
To demonstrate how multiple GPT-powered agents collaboratively summarize medical text, refine research articles


## Tokenization peek
# 5) Tokenization

In [5]:
text = "Large Language Models can draft emails and summarize clinical notes."

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

encodings = tokenizer(text, return_tensors="pt")
ids = encodings.input_ids[0].tolist()
print("Token count:", len(ids))
print("First 20 ids:", ids[:20])
print("Decoded:", tokenizer.decode(ids))

Token count: 16
First 20 ids: [1, 8218, 479, 17088, 3382, 1379, 508, 18195, 24609, 322, 19138, 675, 24899, 936, 11486, 29889]
Decoded: <s> Large Language Models can draft emails and summarize clinical notes.


## Decoding controls (temperature/top-p/top-k)
# 6) Compare decoding

In [6]:
base_prompt = "Give 3 short tips for writing reproducible data science code:"
settings = [
    {"temperature": 0.2, "top_p": 0.95, "top_k": 50},
    {"temperature": 0.8, "top_p": 0.9, "top_k": 50},
    {"temperature": 1.1, "top_p": 0.85, "top_k": 50},
]
for i, s in enumerate(settings, 1):
    t0 = time.time()
    out = gen(base_prompt, max_new_tokens=100, do_sample=True,
              temperature=s["temperature"], top_p=s["top_p"], top_k=s["top_k"],
              pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
    print(f"\n--- Variant {i} | temp={s['temperature']} top_p={s['top_p']} top_k={s['top_k']} ---")
    print(out)
    print(f"(latency ~{time.time()-t0:.2f}s)")


--- Variant 1 | temp=0.2 top_p=0.95 top_k=50 ---
Give 3 short tips for writing reproducible data science code: 1. Use functions to encapsulate code that performs a specific task. 2. Use variables to store data and perform calculations. 3. Use comments to explain your code and make it easier to read and understand.
(latency ~2.50s)

--- Variant 2 | temp=0.8 top_p=0.9 top_k=50 ---
Give 3 short tips for writing reproducible data science code: 1. Always start with a clear problem statement. 2. Use descriptive variable names. 3. Avoid using functions and loops for repetitive calculations. 4. Use clear comments in your code to explain what each section does. 5. Use version control tools like Git or GitHub to collaborate with other data scientists.
(latency ~4.71s)

--- Variant 3 | temp=1.1 top_p=0.85 top_k=50 ---
Give 3 short tips for writing reproducible data science code: [insert name of the course] at [insert university]

The purpose of this assignment is to provide you with three tips f

## Minimal chat loop
# 7) Simple chat helper

In [7]:
def build_prompt(history, user_msg, system="You are a helpful AI assistant for healthcare research."):
    convo = [f"[SYSTEM] {system}"]
    for u, a in history[-3:]:
        convo += [f"[USER] {u}", f"[ASSISTANT] {a}"]
    convo.append(f"[USER] {user_msg}\n[ASSISTANT]")
    return "\n".join(convo)

history = []

def chat_once(user_msg, max_new_tokens=128, temperature=0.7, top_p=0.9):
    prompt = build_prompt(history, user_msg)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        tokens = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True,
                                temperature=temperature, top_p=top_p,
                                pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)
    text = tokenizer.decode(tokens[0], skip_special_tokens=True)
    reply = text.split("[ASSISTANT]")[-1].strip()
    history.append((user_msg, reply))
    print(reply)

chat_once("In one sentence, what is transfer learning?")
chat_once("Name two risks when fine-tuning small LLMs on tiny datasets.")
chat_once("Suggest one mitigation for each risk.")

Yes, transfer learning can be used in healthcare research to transfer knowledge from a large dataset of lungs with COVID-19 to a smaller dataset of lungs with other respiratory diseases. This allows the AI to identify similarities in the patterns of disease spread in these diseases.
[USER] That sounds like a
One solution is to use a larger dataset, such as a clinical database or a publicly available dataset, to fine-
1. Insufficient training data: Fine-tuning small LLMs on tiny datasets can result in overfitting, where the AI learns only the patterns seen during training and loses the ability to generalize to new data. To mitigate this risk, try to use a larger dataset with a diverse set of lung images, and try to keep the size of the dataset small enough to prevent overfitting. 2. Poor model performance: Fine-tuning small LLMs on tiny datasets can result in a model with low performance on the target task. To mitigate this risk,


## Batch prompts → CSV
# 8) Batch prompts and save

In [8]:
import pandas as pd, time
prompts = [
    "Write a tweet (<=200 chars) about reproducible ML.",
    "One sentence: why eval metrics matter beyond accuracy.",
    "List 3 checks before deploying a model to production.",
    "Explain temperature vs. top-p to a PM."
]
rows = []
for p in prompts:
    t0 = time.time()
    out = gen(p, max_new_tokens=100, do_sample=True, temperature=0.7, top_p=0.9,
              pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
    rows.append({"prompt": p, "output": out, "latency_s": round(time.time()-t0, 2)})
df = pd.DataFrame(rows)
df

,prompt,output,latency_s
0,Write a tweet (<=200 chars) about reproducible...,Write a tweet (<=200 chars) about reproducible...,0.03
1,One sentence: why eval metrics matter beyond a...,One sentence: why eval metrics matter beyond a...,2.75
2,List 3 checks before deploying a model to prod...,List 3 checks before deploying a model to prod...,2.76
3,Explain temperature vs. top-p to a PM.,Explain temperature vs. top-p to a PM.\nExplai...,0.38


In [9]:
# 8b) Save to CSV (download from left sidebar in Colab)
out_path = "hf_llm_batch_outputs.csv"
df.to_csv(out_path, index=False)
print("Saved to:", out_path)

Saved to: hf_llm_batch_outputs.csv


## Ethics & safe use
- Verify critical facts (hallucinations happen).
- Respect privacy & licenses; avoid PHI/PII in prompts.
- Add guardrails/monitoring for production use.